In [1]:
%matplotlib inline

In [2]:
from keras.applications import VGG16
#from keras.applications import InceptionResNetV2

Using Theano backend.
Using cuDNN version 5103 on context None
Mapped name None to device cuda: Tesla K80 (30FC:00:00.0)


In [3]:
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, BatchNormalization, Activation, Flatten
from keras.applications.vgg16 import preprocess_input
from keras.optimizers import Adam, Nadam
import numpy as np
from PIL import Image

In [4]:
base_model = VGG16(include_top=False, weights=None, input_tensor=None, input_shape=(3, 224, 224), pooling=None, classes=1000)

In [5]:
train_path = '/home/arman/deep-learning/kaggle-competitions/kaggle-state-farm-notebook/data/train/'
valid_path = '/home/arman/deep-learning/kaggle-competitions/kaggle-state-farm-notebook/data/valid/'
test_path = '/home/arman/deep-learning/kaggle-competitions/kaggle-state-farm-notebook/data/test/'
model_path = '/home/arman/deep-learning/kaggle-competitions/kaggle-state-farm-notebook/'

In [7]:
#base_model.summary()

In [8]:
flat = Flatten()(base_model.layers[-1].output)
new_fc1 = Dense(4096, use_bias=False, kernel_initializer='uniform')(flat)
bn1 = BatchNormalization()(new_fc1)
act1 = Activation('relu')(bn1)
# try adding drop out later

new_fc2 = Dense(4096, use_bias=False, kernel_initializer='uniform')(act1)
bn2 = BatchNormalization()(new_fc2)
act2 = Activation('relu')(bn2)

new_classifier = Dense(10, kernel_initializer='uniform', name='state_farm_classifier')(act2)
#bn3 = BatchNormalization()(new_classifier)
act3 = Activation('softmax')(new_classifier)

inp = base_model.input
out = act3

state_farm_model = Model(inp, out)

for i, layer in enumerate(state_farm_model.layers):
    if i < 19:
        state_farm_model.layers[i].trainable = False
    else:
        state_farm_model.layers[i].trainable = True

In [10]:
#state_farm_model.summary()

In [11]:
for layer in state_farm_model.layers:
    print(layer.name + " " + str(layer.trainable))

input_1 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 False
block5_pool False
flatten_1 True
dense_1 True
batch_normalization_1 True
activation_1 True
dense_2 True
batch_normalization_2 True
activation_2 True
state_farm_classifier True
activation_3 True


In [12]:
state_farm_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
train_gen = image.ImageDataGenerator()
valid_gen = image.ImageDataGenerator()
train_batches = train_gen.flow_from_directory(train_path, target_size=(224,224), shuffle=True, batch_size=8, class_mode='categorical')
valid_batches = valid_gen.flow_from_directory(valid_path, target_size=(224,224), shuffle=True, batch_size=8, class_mode='categorical')

Found 18047 images belonging to 10 classes.
Found 4377 images belonging to 10 classes.


In [ ]:
state_farm_model.fit_generator(train_batches, steps_per_epoch=int(np.ceil(train_batches.samples/32)), epochs=5, 
                    validation_data = valid_batches, validation_steps=int(np.ceil(valid_batches.samples/32)) )

Epoch 1/5
564/564 [==============================] - 150s - loss: 0.0846 - acc: 0.9743 - val_loss: 9.2368 - val_acc: 0.2089
Epoch 2/5
564/564 [==============================] - 151s - loss: 0.0840 - acc: 0.9752 - val_loss: 9.2465 - val_acc: 0.1843
Epoch 3/5
564/564 [==============================] - 152s - loss: 0.0659 - acc: 0.9783 - val_loss: 8.7220 - val_acc: 0.1919
Epoch 4/5
232/564 [===========>..................] - ETA: 75s - loss: 0.0613 - acc: 0.9806

In [21]:
state_farm_model.optimizer.lr.set_value(0.0001)
print(state_farm_model.optimizer.lr.get_value())

9.999999747378752e-05


In [20]:
state_farm_model.save_weights(model_path + 'state_farm.h5')

In [21]:
test_gen = image.ImageDataGenerator()
test_batches = test_gen.flow_from_directory(test_path, target_size=(224,224), shuffle=False, batch_size=8, class_mode=None)

Found 79726 images belonging to 1 classes.


In [22]:
steps = int(np.ceil(test_batches.samples/8))

In [23]:
pred_gen = state_farm_model.predict_generator(test_batches, steps)

In [24]:
pred_gen.shape

(79726, 10)

In [51]:
#Image.open(test_path + test_batches.filenames[3])
#print(test_batches.filenames[0])

In [54]:
#probabilities = pred_gen.clip(min=0.05, max=0.95)

In [25]:
ids = [element[8:] for element in test_batches.filenames]
ids = np.asarray(ids).reshape(-1,1)
print(pred_gen.shape)
print(ids.shape)

(79726, 10)
(79726, 1)


In [26]:
results = np.concatenate([ids,pred_gen], axis=-1)
results.shape

(79726, 11)

In [27]:
submission_file_name = '/home/arman/deep-learning/kaggle-competitions/kaggle-state-farm-notebook/submission1.csv'

In [71]:
#with open(submission_file_name,'wb') as f:
#    np.savetxt(f, results, fmt='%s, %.5f, %.5f, %.5f, %.5f, %.5f, %.5f, %.5f, %.5f, %.5f, %.5f', 
#    header='img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9', comments='')

In [28]:
np.savetxt(submission_file_name, results, fmt='%5s',delimiter=',', header='img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9')